In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARMA
from scipy.stats.distributions import chi2
from sklearn.metrics import r2_score,mean_squared_error
import warnings
from pmdarima.arima import auto_arima
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set()

In [13]:
raw_csv_data = pd.read_csv('SANDWIP.csv')
raw_csv_data.head()
df_comp =raw_csv_data.copy()

In [14]:
df_comp.Date = pd.to_datetime(df_comp.Date, dayfirst = True)
df_comp.set_index("Date",inplace=True)
df_comp

,SURFACE PRESSURE,DEW POINT,TEMPERATURE,PRECIPITATION,MAX WIND SPD,RELATIVE HUMIDITY,MIN TEMP,N_ICE EXTENT,S_ICE EXTENT,HIGH,LOW
Date,,,,,,,,,,,
2017-01-01,101.54,16.69,23.98,0.00,1.88,76.25,17.96,12.643,5.516,0.906,-2.184
2017-02-01,101.53,16.66,24.37,0.01,1.62,76.88,17.60,12.644,5.387,0.846,-2.124
2017-03-01,101.42,16.32,24.60,0.01,2.38,75.31,17.16,12.713,5.232,0.766,-2.064
2017-04-01,101.34,17.26,24.09,0.01,3.11,80.94,17.37,12.954,5.149,0.716,-1.984
2017-05-01,101.31,16.33,24.08,0.01,3.69,78.94,16.62,12.956,5.065,0.666,-1.884
...,...,...,...,...,...,...,...,...,...,...,...
NaT,101.37,14.59,22.06,0.00,3.22,78.75,15.03,12.579,7.441,0.926,-2.844
NaT,101.33,14.44,21.92,0.00,3.04,76.12,15.72,12.582,7.208,0.926,-2.844
NaT,101.37,15.36,21.94,0.00,3.32,83.44,14.87,12.608,7.121,0.926,-2.844


In [15]:
df_comp = df_comp.asfreq('d') #'d' -> daily

ValueError: cannot reindex from a duplicate axis

In [ ]:
df_comp2 = df_comp.copy()
del df_comp['LOW']

In [ ]:
size = int(len(df_comp)*0.8)
df = df_comp.iloc[:size]
df_test = df_comp.iloc[size:]
df2 = df_comp2.iloc[:size]
df2_test = df_comp2.iloc[size:]

In [ ]:
def LLR_test(mod_1, mod_2, DF = 1):
    L1 = mod_1.fit().llf
    L2 = mod_2.fit().llf
    LR = (2*(L2-L1))    
    p = chi2.sf(LR, DF).round(3)
    return p


In [ ]:
sgt.plot_acf(df.HIGH, lags = 40, zero = False)
plt.title("ACF HIGH", size=24)
plt.show()

In [ ]:
sgt.plot_pacf(df.HIGH, lags=40, zero = False, method=('ols'))
plt.title("PACF HIGH", size=24)
plt.show()

In [ ]:
df_test.head()

In [ ]:
df_test.tail()

In [ ]:
start_date = "2020-03-14"
end_date = "2020-12-31"

In [ ]:
df_comp.head()

In [ ]:
mod_auto2 = auto_arima(df.HIGH,exogenous = df[['SURFACE PRESSURE','TEMPERATURE',
                                               'MAX WIND SPD','N_ICE EXTENT']]
                       ,m = 30, max_p = 12, max_q = 12)

In [ ]:
mod_auto2.summary()

In [ ]:
df_auto_pred_pr2 = pd.DataFrame(mod_auto2.predict(n_periods = len(df_test[start_date:end_date]),exogenous = df_test[['SURFACE PRESSURE','TEMPERATURE',
                                               'MAX WIND SPD','S_ICE EXTENT']][start_date:end_date]),
                               index = df_test[start_date:end_date].index)

In [ ]:
df_auto_pred_pr2[start_date:end_date].plot(figsize = (20,5), color = "red")
df_test.HIGH[start_date:end_date].plot(color = "blue")

In [ ]:
acc =r2_score(df_test.HIGH[start_date:end_date],df_auto_pred_pr2[start_date:end_date])
print("Testing Score: ",acc)